In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from models import (TorchComplexMLP, ImaginaryDimensionAdder, 
                    cplx2tensor, ComplexTorchMLP, complex_mse, TanhProb)
from preprocess import *

# Model selection
# from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

# Fancy optimizers
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew

Running Python 3.9.6
You can use npar for np.array


In [2]:
data = pickle_load('../deephpms_data/KdV_simple2.pkl')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['u'])

X_sol, T_sol = np.meshgrid(x, t)

x_star = X_sol.flatten()[:,None]
t_star = T_sol.flatten()[:,None]

X_star = np.hstack((x_star, t_star))
if Exact.shape[1]==X_sol.shape[0] and Exact.shape[0]==X_sol.shape[1]:
    Exact = Exact.T
u_star = Exact.flatten()[:,None]

# DATA_PATH = '../PDE_FIND_experimental_datasets/kuramoto_sivishinky.mat'
# X, T, Exact = space_time_grid(data_path=DATA_PATH, real_solution=True, uniform=True, x_limit=None, t_limit=None)
# X_star, u_star = get_trainable_data(X, T, Exact)

# Bound
ub = X_star.max(axis=0)
lb = X_star.min(axis=0)

# For identification
N = 500
# idx = np.arange(N)
idx = np.random.choice(X_star.shape[0], N, replace=False)
X_train = X_star[idx,:]
u_train = u_star[idx,:]
print("Training with", N, "samples")

noise_intensity = 0.01
noisy_xt, noisy_labels = False, False
if noisy_xt: X_train = perturb(X_train, noise_intensity); print("Noisy (x, t)")
else: print("Clean (x, t)")
if noisy_labels: u_train = perturb(u_train, noise_intensity); print("Noisy labels")
else: print("Clean labels")

# Unsup data
include_N_res = True
if include_N_res:
    N_res = N
    idx_res = np.array(range(X_star.shape[0]-1))[~idx]
    idx_res = np.random.choice(idx_res.shape[0], N_res, replace=True)
    X_res = X_star[idx_res, :]
    print(f"Training with {N_res} unsup samples")
    X_u_train = np.vstack([X_train, X_res])
    u_train = np.vstack([u_train, torch.rand(X_res.shape[0], 1) - 1000])
    # del X_res
else: print("Not including N_res")
    
# Convert to torch.tensor
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names
feature_names=('uf', 'u_x', 'u_xx', 'u_xxx')

Loaded from ../deephpms_data/KdV_simple2.pkl
Training with 500 samples
Training with 500 unsup samples


In [3]:
class Network(nn.Module):
    def __init__(self, model, index2features=None, scale=False, lb=None, ub=None):
        super(Network, self).__init__()
        # pls init the self.model before
        self.model = model
        # For tracking, the default tup is for the burgers' equation.
        self.index2features = index2features
        print("Considering", self.index2features)
        self.diff_flag = diff_flag(self.index2features)
        self.uf = None
        self.scale = scale
        self.lb, self.ub = lb, ub
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def forward(self, x, t):
        if not self.scale: self.uf = self.model(torch.cat([x, t], dim=1))
        else: self.uf = self.model(self.neural_net_scale(torch.cat([x, t], dim=1)))
        return self.uf
    
    def get_selector_data(self, x, t):
        uf = self.forward(x, t)
        u_t = self.gradients(uf, t)[0]
        
        ### PDE Loss calculation ###
        # Without calling grad
        derivatives = []
        for t in self.diff_flag[0]:
            if t=='uf': derivatives.append(uf)
            elif t=='x': derivatives.append(x)
        # With calling grad
        for t in self.diff_flag[1]:
            out = uf
            for c in t:
                if c=='x': out = self.gradients(out, x)[0]
                elif c=='t': out = self.gradients(out, t)[0]
            derivatives.append(out)
        
        return torch.cat(derivatives, dim=1), u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    def neural_net_scale(self, inp):
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [4]:
class AttentionSelectorNetwork(nn.Module):
    def __init__(self, layers, prob_activation=torch.sigmoid, bn=None, reg_intensity=7.5e-4): #5e-4 and training with 400 epochs are also OK!
        super(AttentionSelectorNetwork, self).__init__()
        # Nonlinear model, Training with PDE reg.
        assert len(layers) > 1
        self.linear1 = nn.Linear(layers[0], layers[0])
        self.prob_activation = prob_activation
        self.nonlinear_model = TorchMLP(dimensions=layers, activation_function=nn.Tanh, bn=bn, dropout=nn.Dropout(p=0.1))
        self.latest_weighted_features = None
        self.th = (1/layers[0])-(1e-10)
        self.reg_intensity = reg_intensity
        self.w = torch.tensor([1.0, 1.0, 2.0, 3.0])/10.0
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, inn):
        return self.nonlinear_model(inn*(F.threshold(self.weighted_features(inn), self.th, 0.0)))
    
    def weighted_features(self, inn):
        self.latest_weighted_features = self.prob_activation(self.linear1(inn)).mean(axis=0)
        return self.latest_weighted_features
    
    def loss(self, X_input, y_input):
        ut_approx = self.forward(X_input)
        mse_loss = F.mse_loss(ut_approx, y_input, reduction='mean')
        reg_term = F.relu(self.latest_weighted_features-self.th)
        
        l1 = mse_loss
        l2 = torch.norm(reg_term, p=0)+torch.dot(self.w, reg_term)
        return l1+self.reg_intensity*(l2)

class SemiSupModel(nn.Module):
    def __init__(self, network, selector, normalize_derivative_features=False, mini=None, maxi=None):
        super(SemiSupModel, self).__init__()
        self.network = network
        self.selector = selector
        self.normalize_derivative_features = normalize_derivative_features
        self.mini = mini
        self.maxi = maxi
        
    def forward(self, X_u_train):
        X_selector, y_selector = self.network.get_selector_data(*dimension_slicing(X_u_train))
        if self.normalize_derivative_features:
            X_selector = (X_selector-self.mini)/(self.maxi-self.mini)
        unsup_loss = self.selector.loss(X_selector, y_selector)
        return self.network.uf, unsup_loss

In [5]:
### Version with normalized derivatives ###
# pretrained_state_dict = None
pretrained_state_dict = cpu_load("./saved_path_inverse_small_KdV/simple2_semisup_model_state_dict_250labeledsamples.pth")
use_pretrained_weights = True
lets_pretrain = False

semisup_model = SemiSupModel(network=Network(
                                    model=TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1],
                                                   activation_function=nn.Tanh,
                                                   bn=nn.LayerNorm, dropout=None),
                                    index2features=feature_names, scale=True, lb=lb, ub=ub),
                            selector=AttentionSelectorNetwork([len(feature_names), 50, 50, 1], prob_activation=TanhProb(), bn=nn.LayerNorm),
                            normalize_derivative_features=False,
                            mini=None,
                            maxi=None)

if use_pretrained_weights:
    print("Use pretrained weights")
    semisup_model.load_state_dict(pretrained_state_dict)
    referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_train))
    semisup_model.mini = torch.min(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
    semisup_model.maxi = torch.max(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
#     semisup_model.mini = tmp.min(axis=0)[0].requires_grad_(False)
#     semisup_model.maxi = tmp.max(axis=0)[0].requires_grad_(False)

Using old implementation of TorchMLP. See models.py for more new model-related source code.
Considering ('uf', 'u_x', 'u_xx', 'u_xxx')
Using old implementation of TorchMLP. See models.py for more new model-related source code.
Use pretrained weights


In [6]:
if lets_pretrain:
    print("Pretraining")
    pretraining_optimizer = LBFGSNew(semisup_model.network.parameters(),
                                     lr=1e-1, max_iter=300,
                                     max_eval=int(300*1.25), history_size=150,
                                     line_search_fn=True, batch_mode=False)

    semisup_model.network.train()    
    for i in range(120):
        def pretraining_closure():
            global N, X_u_train, u_train
            if torch.is_grad_enabled():
                pretraining_optimizer.zero_grad()
            # Only focusing on first [:N, :] elements
            mse_loss = F.mse_loss(semisup_model.network(*dimension_slicing(X_train[:N, :])), u_train[:N, :])
            if mse_loss.requires_grad:
                mse_loss.backward(retain_graph=False)
            return mse_loss

        pretraining_optimizer.step(pretraining_closure)
            
        if (i%10)==0:
            l = pretraining_closure()
            curr_loss = l.item()
            print("Epoch {}: ".format(i), curr_loss)

            # Sneak on the test performance...
            semisup_model.network.eval()
            test_performance = F.mse_loss(semisup_model.network(*dimension_slicing(X_star)).detach(), u_star).item()
            string_test_performance = scientific2string(test_performance)
            print('Test MSE:', string_test_performance)
    
#     if best_state_dict is not None: semisup_model.load_state_dict(best_state_dict)
    print("Computing derivatives features")
    semisup_model.eval()
    referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_train))
    semisup_model.mini = torch.min(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
    semisup_model.maxi = torch.max(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)

#### Joint training

In [7]:
def pcgrad_closure(return_list=False):
    global N, X_train, u_train
    predictions, unsup_loss = semisup_model(X_train)
    losses = [F.mse_loss(predictions[:N, :], u_train[:N, :]), unsup_loss]
    updated_grads = []
    
    for i in range(2):
        optimizer.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in semisup_model.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(semisup_model.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    if not return_list: return sum(losses)
    else: return losses

In [8]:
# Joint training
optimizer = MADGRAD([{'params':semisup_model.network.parameters()}, {'params':semisup_model.selector.parameters()}], lr=1e-6)
optimizer.param_groups[0]['lr'] = 1e-7 # Used to be 1e-11.
optimizer.param_groups[1]['lr'] = 1e-2

# Use ~idx to sample adversarial data points
for i in range(500):
    semisup_model.train()
    optimizer.step(pcgrad_closure)
    loss = pcgrad_closure(return_list=True)
    if i%25==0: print(loss)

[tensor(0.0003, grad_fn=<MseLossBackward>), tensor(18.6473, grad_fn=<AddBackward0>)]
[tensor(0.0096, grad_fn=<MseLossBackward>), tensor(0.0932, grad_fn=<AddBackward0>)]
[tensor(0.0070, grad_fn=<MseLossBackward>), tensor(0.0245, grad_fn=<AddBackward0>)]
[tensor(0.0038, grad_fn=<MseLossBackward>), tensor(0.0150, grad_fn=<AddBackward0>)]
[tensor(0.0019, grad_fn=<MseLossBackward>), tensor(0.0110, grad_fn=<AddBackward0>)]
[tensor(0.0011, grad_fn=<MseLossBackward>), tensor(0.0077, grad_fn=<AddBackward0>)]
[tensor(0.0006, grad_fn=<MseLossBackward>), tensor(0.0056, grad_fn=<AddBackward0>)]
[tensor(0.0004, grad_fn=<MseLossBackward>), tensor(0.0046, grad_fn=<AddBackward0>)]
[tensor(0.0003, grad_fn=<MseLossBackward>), tensor(0.0042, grad_fn=<AddBackward0>)]
[tensor(0.0002, grad_fn=<MseLossBackward>), tensor(0.0039, grad_fn=<AddBackward0>)]
[tensor(0.0002, grad_fn=<MseLossBackward>), tensor(0.0038, grad_fn=<AddBackward0>)]
[tensor(0.0002, grad_fn=<MseLossBackward>), tensor(0.0037, grad_fn=<AddBack

In [9]:
# Fine-tuning the solver network
f_opt = torch.optim.LBFGS(semisup_model.network.parameters(), lr=1e-1, max_iter=300, max_eval=int(1.25*300), history_size=300)

def finetuning_closure():
    global N, X_train, u_train
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # the solver network only consider the first N samples.
    loss = F.mse_loss(semisup_model.network(*dimension_slicing(X_train[:N, :])), u_train[:N, :])
    if loss.requires_grad: loss.backward(retain_graph=True)
    return loss

semisup_model.network.train()
semisup_model.selector.eval()

for i in range(50):
    f_opt.step(finetuning_closure)
    if i%10==0:
        loss = finetuning_closure()
        print(loss.item())

1.0604286444504396e-06
4.343440025422751e-07
4.3369294644435286e-07
4.3369294644435286e-07
4.3369294644435286e-07


In [10]:
X_selector, y_selector = semisup_model.network.get_selector_data(*dimension_slicing(X_train))
# X_selector = (X_selector - semisup_model.mini)/(semisup_model.maxi-semisup_model.mini)

In [11]:
f_opt = torch.optim.LBFGS(semisup_model.selector.parameters(), lr=1e-1, max_iter=500, max_eval=int(1.25*500), history_size=500)

def finetuning_closure():
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # Am I forget to normalize the derivative features?, NVM
#     loss = F.mse_loss(semisup_model.selector(X_selector), y_selector)
    loss = semisup_model.selector.loss(X_selector, y_selector)
    if loss.requires_grad: loss.backward(retain_graph=True)
    return loss

semisup_model.network.eval()
semisup_model.selector.train()

max_it = 10
for i in range(max_it):
    f_opt.step(finetuning_closure)
    
    if i%1==0 or i==max_it-1:
        loss = finetuning_closure()
        print(loss.item())
        print(np.argsort(semisup_model.selector.latest_weighted_features.detach().numpy()))

0.0028705946169793606
[2 3 0 1]
0.00288115325383842
[2 3 0 1]
0.002877661259844899
[2 3 0 1]
0.002867615781724453
[2 3 0 1]
0.0028592045418918133
[2 3 0 1]
0.002871139207854867
[2 3 0 1]
0.0028576015029102564
[2 3 0 1]
0.002881326712667942
[2 3 0 1]
0.0028710784390568733
[2 3 0 1]
0.002874873112887144
[2 3 0 1]


In [12]:
# n_test = X_star.shape[0]
n_test = 50000
idx_test = np.arange(n_test)
referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_star[idx_test, :]))
# referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_train))

In [13]:
referenced_derivatives = to_numpy(referenced_derivatives); u_t = to_numpy(u_t)

alpha = 1
const_range = (-1.5, 1.5)

X_input = referenced_derivatives
y_input = u_t

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_input = poly.fit_transform(X_input)

poly_feature_names = poly.get_feature_names(feature_names)
for i, f in enumerate(poly_feature_names):
    poly_feature_names[i] = f.replace(" ", "*")

In [14]:
# Set normalize=1
w = TrainSTRidge(X_input[:, :], y_input, 1e-6, 1000, maxit=100, STR_iters=100, split=0.8, l0_penalty=None, normalize=2)
print("PDE derived using STRidge")
print_pde(w, poly_feature_names[:])

PDE derived using STRidge
u_t = (-0.948363 +0.000000i)u_xxx
    + (-5.835543 +0.000000i)uf*u_x
   


In [21]:
semisup_model.selector.latest_weighted_features.detach().numpy()

array([0.36536175, 0.5783518 , 0.248983  , 0.2742597 ], dtype=float32)

In [22]:
save(semisup_model, "./saved_path_inverse_small_KdV/simple2_semisup_model_state_dict_500labeledsamples500unlabeledsamples_tanhV2.pth")

In [ ]:
# Clean all (500 labels are enough.)
# PDE derived using STRidge 
# u_t = (-0.856551 +0.000000i)u_xxx
#     + (-5.615004 +0.000000i)uf*u_x
# V2
# u_t = (-0.948363 +0.000000i)u_xxx
#     + (-5.835543 +0.000000i)uf*u_x

# Clean (x, t) but noisy labels (500 labels are enough.) 
# PDE derived using STRidge
# u_t = (-0.998013 +0.000000i)u_xxx
#     + (-6.004286 +0.000000i)uf*u_x

# Noisy (x, t) and noisy labels (2000 labels)
# PDE derived using STRidge
# u_t = (-0.846196 +0.000000i)u_xxx
#     + (-5.573660 +0.000000i)uf*u_x